In [1]:
from transformers import AutoTokenizer, AutoModel

In [2]:
tokenizer = AutoTokenizer.from_pretrained("Geotrend/distilbert-base-da-cased")
model = AutoModel.from_pretrained("Geotrend/distilbert-base-da-cased")